# Examples usage of the majority voting I/O processor

This notebook should eventually turn into documentation.

In [1]:
from granite_io import make_backend
from granite_io.io import make_io_processor
from granite_io.io.base import ChatCompletionInputs
from granite_io.io.voting import MajorityVotingProcessor, integer_normalizer

In [2]:
# Connect to a backend running on localhost.
port = 11434
url = f"http://localhost:{port}/v1"
model_name = "granite3.2:2b"
backend = make_backend(
    "openai",
    {
        "model_name": model_name,
        "openai_base_url": url,
    },
)

# Uncomment to run inference on a local Transformers backend.
# backend =  make_backend(
#     "transformers",
#     {
#         "model_name": "ibm-granite/granite-3.2-2b-instruct",
#     },
# )

In [3]:
# Ask a question with an integral answer
completion_inputs = ChatCompletionInputs(
    messages=[
        {
            "role": "user",
            "content": "What is 234651 + 13425?\nAnswer with just a number please.",
        }
    ],
    thinking=True,
    generate_inputs={"n": 1, "max_tokens": 1024},
)
completion_inputs

ChatCompletionInputs(messages=[UserMessage(content='What is 234651 + 13425?\nAnswer with just a number please.', role='user')], tools=[], generate_inputs=GenerateInputs(prompt=None, model=None, best_of=None, timeout=None, temperature=None, top_p=None, n=1, stream=None, stream_options=None, stop=None, max_tokens=1024, presence_penalty=None, frequency_penalty=None, logit_bias=None, user=None, seed=None, extra_headers=None), thinking=True)

In [4]:
# Run the question through the base model
base_processor = make_io_processor("Granite 3.2", backend=backend)
results = base_processor.create_chat_completion(completion_inputs)
for r in results.results:
    print(r.next_message.content)

110006


In [5]:
# Wrap the base model's I/O processor in a majority voting I/O processor.
voting_processor = MajorityVotingProcessor(
    base_processor, integer_normalizer, samples_per_completion=100
)
results = voting_processor.create_chat_completion(completion_inputs)
for r in results.results:
    print(r.next_message.content)

248076


In [6]:
# What's the actual answer?
234651 + 13425

248076